In [1]:
'''
File names can't contain special characters or spaces at the moment
'''

## PACKAGES

In [2]:
# import the necessary packages
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os 
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pdf2image
import glob
import subprocess

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import itertools
from itertools import chain

## PARAMETERS

In [3]:
## READ IN MAPPING FILE 
mapping = pd.read_excel(r'C:\Users\Adam\Python Scripts\invoice_mapping\type_map.xlsx')
mapping = pd.DataFrame(mapping)

## CREATE TUPLE WITH INVOICE TYPES
invoice_types = mapping['Type'].tolist() 
invoice_types = tuple(invoice_types)

## CREATE TUPLE OF EXAMPLE INVOICES 
invoice_examples = mapping['Example'].tolist() 
invoice_examples = tuple(invoice_examples)

# get results to this directory 
result_dir = 'C:\\Users\\Adam\\Python Scripts\\feldolgozott_kepek\\'

## BUILD IMAGE SIMILARITY MATRIX 

#### LOOPY LOOP 

In [4]:
# SET WD TO DIRECTORY TO LOOP THROUGH  
os.chdir('C:\\Users\\Adam\\Python Scripts\\feldolgozott_kepek\\')

# create list with all files 
files = os.listdir()

In [5]:
# CREATE ALL POSSIBLE PERMUTATIONS OF IMAGES FOR COMPATING 
#permutations = [list(zip(x,invoice_examples)) for x in itertools.permutations(files,len(invoice_examples))]
#perm = sorted(list(set(list(chain(*permutations)))), reverse=True)

In [6]:
perm = list(itertools.product(invoice_examples, files))

In [ ]:
perm

In [7]:
invoice_compare_to = list()
invoice_compared= list()
similarity= list()
similarity_matrix = pd.DataFrame()

In [8]:
for i in range(len(perm)):
    #print(perm[i])
    
    test = perm[i]
    
    ## READ IMAGE I 

    # convert image to numeric matrix
    original = cv2.imread(test[0])
    # convert to greyscale
    original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    ## READ IMAGE II 

    # convert image to numeric matrix
    original2 = cv2.imread(test[1])
    # convert to greyscale
    original2 = cv2.cvtColor(original2, cv2.COLOR_BGR2GRAY)
    
    
    ## CONVERT IMAGES TO SAME SIZE
    (H, W) = original.shape
    original2 = cv2.resize(original2, (W, H))

    s = ssim(original, original2)

    ## STORE RESULTS 
    invoice1 = test[0]
    invoice2= test[1]
    similarity_score= s
    
    invoice_compare_to.append(invoice1)
    invoice_compared.append(invoice2)
    similarity.append(similarity_score)
    

In [9]:
sim_df = pd.DataFrame(
    {'Szamla': invoice_compare_to,
     'Szamlahoz_hasonlitott': invoice_compared,
     'Hasonlosag': similarity
    })

In [10]:
sim_df

,Szamla,Szamlahoz_hasonlitott,Hasonlosag
0,t1_szamla_1.tiff,E-KBOSS-2020-58455.tiff,0.664223
1,t1_szamla_1.tiff,E-KBOSS-2020-78468.tiff,0.664620
2,t1_szamla_1.tiff,E-KBOSS-2020-98913.tiff,0.663967
3,t1_szamla_1.tiff,ST-2020-44.tiff,0.663906
4,t1_szamla_1.tiff,t1_szamla_1.tiff,1.000000
...,...,...,...
95,E-KBOSS-2020-58455.tiff,t3_szamla_4.tiff,0.903614
96,E-KBOSS-2020-58455.tiff,t4_szamla_1.tiff,0.816413
97,E-KBOSS-2020-58455.tiff,t4_szamla_2.tiff,0.816413
98,E-KBOSS-2020-58455.tiff,t4_szamla_3.tiff,0.816413


## PROCESS IMAGE CLASSES 

In [11]:
# CREATE FOLDERS FOR EACH TYPE 

# create as many folders as invoice types 
for i in range(len(invoice_types)): 
    create = result_dir+invoice_types[i]
    if not os.path.exists(create):
        os.makedirs(create)
    print('DIRECTORY CREATED:')
    print(create)

DIRECTORY CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type1
DIRECTORY CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type2
DIRECTORY CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type3
DIRECTORY CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type4
DIRECTORY CREATED:
C:\Users\Adam\Python Scripts\feldolgozott_kepek\Type5


In [12]:
sim_df

,Szamla,Szamlahoz_hasonlitott,Hasonlosag
0,t1_szamla_1.tiff,E-KBOSS-2020-58455.tiff,0.664223
1,t1_szamla_1.tiff,E-KBOSS-2020-78468.tiff,0.664620
2,t1_szamla_1.tiff,E-KBOSS-2020-98913.tiff,0.663967
3,t1_szamla_1.tiff,ST-2020-44.tiff,0.663906
4,t1_szamla_1.tiff,t1_szamla_1.tiff,1.000000
...,...,...,...
95,E-KBOSS-2020-58455.tiff,t3_szamla_4.tiff,0.903614
96,E-KBOSS-2020-58455.tiff,t4_szamla_1.tiff,0.816413
97,E-KBOSS-2020-58455.tiff,t4_szamla_2.tiff,0.816413
98,E-KBOSS-2020-58455.tiff,t4_szamla_3.tiff,0.816413


In [13]:
invoice_types

('Type1', 'Type2', 'Type3', 'Type4', 'Type5')

In [14]:
invoice_examples

('t1_szamla_1.tiff',
 't2_szamla_1.tiff',
 't3_szamla_1.tiff',
 't4_szamla_1.tiff',
 'E-KBOSS-2020-58455.tiff')

In [15]:
# Sort invoices into directory with very similar benchmark invoce 

for i in range(len(invoice_types)):
    
    # create df for invoice type 
    type1 = sim_df.loc[(sim_df['Szamla'] == invoice_examples[i])  ]
    
    # filter for similar invoices only
    move = type1.loc[type1['Hasonlosag'] > 0.90]
    
    # create list of filenames to move to type's folder
    files_to_move = move['Szamlahoz_hasonlitott']
    files_to_move = list(files_to_move)
    
    # move files to appropriate folder 
    dest_folder =  os.getcwd() + '\\' +  invoice_types[i]
    #files = files_to_move

    for f in files_to_move:
        shutil.move(f, dest_folder)

FileNotFoundError: [Errno 2] No such file or directory: 't3_szamla_1.tiff'

In [ ]:
sim_df